In [ ]:
import pickle
import numpy as np

In [ ]:
!pwd

In [ ]:
filename = "data/ntu/one_shot/sample_label.pkl"
filename_sample_data = "data/ntu/one_shot/sample_data_joint.npy"
filename_train_data = "data/ntu/one_shot/train_data_joint.npy"
filename_val_data = "data/ntu/one_shot/val_data_joint.npy"

In [ ]:
def read_pickle(filename):
    with open(filename, "rb") as f:
        data = pickle.load(f)
    return data

## Sample data

In [ ]:
sample_joint_data = np.load(filename_sample_data)
sample_joint_data.shape

## Validation Data

In [ ]:
val_joint_data = np.load(filename_val_data)
val_joint_data.shape

In [ ]:
val_joint_data[0].shape

In [ ]:
val_joint_data[0].shape

In [ ]:
# samples = read_pickle(filename_val_data)

# GCN

In [ ]:
import  torch
from    torch import nn
from    torch.nn import functional as F
# from    layer import GraphConvolution

In [ ]:
def sparse_dropout(x, rate, noise_shape):
    """
    :param x:
    :param rate:
    :param noise_shape: int scalar
    :return:
    """
    random_tensor = 1 - rate
    random_tensor += torch.rand(noise_shape).to(x.device)
    dropout_mask = torch.floor(random_tensor).byte()
    i = x._indices() # [2, 49216]
    v = x._values() # [49216]

    # [2, 4926] => [49216, 2] => [remained node, 2] => [2, remained node]
    i = i[:, dropout_mask]
    v = v[dropout_mask]

    out = torch.sparse.FloatTensor(i, v, x.shape).to(x.device)

    out = out * (1./ (1-rate))

    return out


def dot(x, y, sparse=False):
    if sparse:
        res = torch.sparse.mm(x, y)
    else:
        res = torch.mm(x, y)

    return res

In [ ]:
class GraphConvolution(nn.Module):


    def __init__(self, input_dim, output_dim, num_features_nonzero,
                 dropout=0.,
                 is_sparse_inputs=False,
                 bias=False,
                 activation = F.relu,
                 featureless=False):
        super(GraphConvolution, self).__init__()


        self.dropout = dropout
        self.bias = bias
        self.activation = activation
        self.is_sparse_inputs = is_sparse_inputs
        self.featureless = featureless
        self.num_features_nonzero = num_features_nonzero

        self.weight = nn.Parameter(torch.randn(input_dim, output_dim))
        self.bias = None
        if bias:
            self.bias = nn.Parameter(torch.zeros(output_dim))


    def forward(self, inputs):
        # print('inputs:', inputs)
        x, support = inputs

        if self.training and self.is_sparse_inputs:
            x = sparse_dropout(x, self.dropout, self.num_features_nonzero)
        elif self.training:
            x = F.dropout(x, self.dropout)

        # convolve
        if not self.featureless: # if it has features x
            if self.is_sparse_inputs:
                xw = torch.sparse.mm(x, self.weight)
            else:
                xw = torch.mm(x, self.weight)
        else:
            xw = self.weight

        out = torch.sparse.mm(support, xw)

        if self.bias is not None:
            out += self.bias

        return self.activation(out), support

In [ ]:
class GCN(nn.Module):


    def __init__(self, input_dim, output_dim, num_features_nonzero):
        super(GCN, self).__init__()

        hidden_layers = 16
        drop_out_rate = 0.5
        self.input_dim = input_dim # 1433
        self.output_dim = output_dim

        print('input dim:', input_dim)
        print('output dim:', output_dim)
        print('num_features_nonzero:', num_features_nonzero)


        self.layers = nn.Sequential(GraphConvolution(self.input_dim, hidden_layers, num_features_nonzero,
                                                     activation=F.relu,
                                                     dropout=drop_out_rate,
                                                     is_sparse_inputs=True),

                                    GraphConvolution(hidden_layers, output_dim, num_features_nonzero,
                                                     activation=F.relu,
                                                     dropout=drop_out_rate,
                                                     is_sparse_inputs=False),

                                    )

    def forward(self, inputs):
        x, support = inputs

        x = self.layers((x, support))

        return x

    def l2_loss(self):

        layer = self.layers.children()
        layer = next(iter(layer))

        loss = None

        for p in layer.parameters():
            if loss is None:
                loss = p.pow(2).sum()
            else:
                loss += p.pow(2).sum()

        return loss

In [ ]:
val_joint_data[0].shape

In [ ]:
input_dim = val_joint_data[0].shape[0] *  val_joint_data[0].shape[1] *  val_joint_data[0].shape[2] *  val_joint_data[0].shape[3]

In [ ]:
a = GCN(input_dim, 100, 0)

In [ ]:
data = torch.from_numpy(val_joint_data[0])

In [ ]:
a((data, 200))